<a href="https://colab.research.google.com/github/Namo-Bang/55_taunt/blob/main/55_taunt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 원클릭 55도발 메이커
### **편집기 필요없어도 가능합니다.**

코드 수정: [Namo-Bang](https://github.com/Namo-Bang/55_taunt)

*모든 코드는 KeepSOBP님의 다메다네 메이커를 참고하여 작성되었습니다.

참고 코드: [KeepSOBP](https://github.com/KeepSOBP/dame) - 20201213 버전(다메다네 메이커)
 
원본 : [AliaksandrSiarohin](https://github.com/AliaksandrSiarohin/first-order-model) - 20200901 버전
 
---

In [ ]:
#@markdown ### <- 플레이 버튼 한번으로 자신만의 55도발을 만들어 보세요!
 
# Check gpu runtime
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  # 여기서 에러가 발생하시면, GPU 런타임이 아니라서 그런겁니다.
  # 구글 부계로 다시 시도해보세요.
  raise SystemError("GPU 런타임이 아닙니다!")
 
# upload image
print("이미지를 올려주세요, png jpg 지원!")
from google.colab import files
from IPython.display import clear_output
uploaded = files.upload()
for fn in uploaded.keys():
  uploadimage = fn
  clear_output(wait=True)
 
import os
import time
if not os.path.isdir("/content/55taunt/"):
  print("이전 작업 초기화 작업 & 설치중")
  !git clone https://github.com/AliaksandrSiarohin/first-order-model
  !wget https://github.com/Namo-Bang/55_taunt/releases/download/55tauntsource/55taunt.zip
  !wget https://github.com/Namo-Bang/55_taunt/releases/download/55tauntsource/55_taunt_audio.mp3
  !unzip -o 55taunt.zip -d /content/55taunt
  !rm -rf 55taunt.zip
  clear_output(wait=True)
 
 # prevent 0 byte error
if os.path.isfile("/content/complete.mp4"):
  os.remove("complete.mp4")
 
# make damedane
import sys
sys.path.insert(1, '/content/first-order-model/')
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
import warnings
from demo import load_checkpoints, make_animation
from skimage import img_as_ubyte
 
print("딥페이크 작업중")
warnings.filterwarnings("ignore")
 
source_image = imageio.imread(uploadimage)
driving_video = imageio.get_reader('/content/55taunt/55_taunt.mp4')
 
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
 
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
 
generator, kp_detector = load_checkpoints(config_path='/content/first-order-model/config/vox-256.yaml', 
                            checkpoint_path='/content/55taunt/vox-cpk.pth.tar')
 
clear_output(wait=True)
 
# make video
print("영상 제작, 다운로드 진행중")
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
 
imageio.mimsave('original.mp4', [img_as_ubyte(frame) for frame in predictions])
clear_output(wait=True)
 
# 영상 2.4배, 음원 삽입 (ffmpeg 이용)
print("영상 편집중")
!ffmpeg -i original.mp4 -r 60 -vf "setpts=(PTS-STARTPTS)/2.4" 3x.mp4
!ffmpeg -i 3x.mp4 -i 55_taunt_audio.mp3 -map 0:v -map 1:a -c:v copy -shortest complete.mp4
clear_output(wait=True)
 
print("영상 다운중")
files.download('complete.mp4')
 
os.remove(uploadimage)
os.remove("3x.mp4")
os.remove("original.mp4")

 
---
### 또 만들고 싶어요!
플레이 버튼을 다시 클릭하시면 됩니다.
 
### 사진을 자르고 싶어요!
https://www.iloveimg.com/ko/crop-image 에 들어가셔서 자르시면 됩니다, 크기는 아무 정사각형 크기로 잘라주세요!
 
### GPU 런타임을 쓸 수 없어요!
 
`현재 Colab의 사용량 제한으로 인해 GPU에 연결할 수 없습니다.`
 
라면서 gpu 런타임에 연결할 수 없으시면
 
구글 부계를 이용하시거나, 나중에 다시 시도해보시길 바랍니다.
---
 
 
**여기에는 바이러스가 없습니다**
 
이 colab 링크를 이용하여 유튜브에 *다메다네 튜토리얼* 영상을 만드는 행동은 **금지되어 있습니다.**
 
튜토리얼은 [제 채널에서 보시는걸로!](https://www.youtube.com/channel/UCl7UlXHLVg2hs7VbrZs0y3Q)